In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf
import urllib.request

In [5]:
train_ner_df = pd.read_csv('ner_train_data.csv')
test_ner_df = pd.read_csv('ner_test_data.csv')

In [6]:
train_ner_df.head()

,Sentence,Tag
0,"정은 씨를 힘들게 한 가스나그, 가만둘 수 없겠죠 .",PER-B O O O O O O O O
1,▶ 쿠마리 한동수가 말하는 '가넷 & 에르덴',O PER-B PER-I O PER-B O PER-B
2,슈나이더의 프레젠테이션은 말 청중을 위한 특별한 쇼다 .,PER-B O O CVL-B O O O O
3,지구 최대 연료탱크 수검 회사 구글이 연내 22명 안팎의 인력을 갖춘 연구개발(R&...,O O TRM-B O O ORG-B DAT-B NUM-B O O O ORG-B LO...
4,5. <10:00:TI_HOUR> 도이치증권대 <0:1:QT_SPORTS> 연예오락...,NUM-B O ORG-B O ORG-B


In [7]:
test_ner_df.head()

,Sentence,Tag
0,"라티은-원윤정, 휘닉스파크클래식 프로골퍼",PER-B EVT-B CVL-B
1,5원으로 맺어진 애인까지 돈이라는 민감한 원자재를 통해 현대인의 물질만능주의를 꼬집...,NUM-B O O O O O O O O O O O FLD-B O
2,-날로 삼키면 맛이 어떤지 일차 드셔보시겠어요 .,O O O O NUM-B O O
3,"-네, 지었습니다 .",O O O
4,◇신규 투자촉진에 방점=이번 접속료 조정결과에서 눈에 띄는 지점은 WCDMA/HSD...,O O O O O O O O TRM-B O TRM-B TRM-I ORG-B O TR...


In [8]:
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
train_data_label = [tag.split() for tag in train_ner_df['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]


In [9]:
print(train_data_sentence[2])
print(train_data_label[2])

['슈나이더의', '프레젠테이션은', '말', '청중을', '위한', '특별한', '쇼다', '.']
['PER-B', 'O', 'O', 'CVL-B', 'O', 'O', 'O', 'O']


In [10]:
# 개체명 태깅 정보의 종류 확인
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf=8')]
print(labels)

['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']


In [11]:
tag_to_index = {tag : index for index, tag in enumerate(labels)}
index_to_tag = {index : tag for index, tag in enumerate(labels)}

In [12]:
print(tag_to_index)
print()
print(index_to_tag)

{'O': 0, 'PER-B': 1, 'PER-I': 2, 'FLD-B': 3, 'FLD-I': 4, 'AFW-B': 5, 'AFW-I': 6, 'ORG-B': 7, 'ORG-I': 8, 'LOC-B': 9, 'LOC-I': 10, 'CVL-B': 11, 'CVL-I': 12, 'DAT-B': 13, 'DAT-I': 14, 'TIM-B': 15, 'TIM-I': 16, 'NUM-B': 17, 'NUM-I': 18, 'EVT-B': 19, 'EVT-I': 20, 'ANM-B': 21, 'ANM-I': 22, 'PLT-B': 23, 'PLT-I': 24, 'MAT-B': 25, 'MAT-I': 26, 'TRM-B': 27, 'TRM-I': 28}

{0: 'O', 1: 'PER-B', 2: 'PER-I', 3: 'FLD-B', 4: 'FLD-I', 5: 'AFW-B', 6: 'AFW-I', 7: 'ORG-B', 8: 'ORG-I', 9: 'LOC-B', 10: 'LOC-I', 11: 'CVL-B', 12: 'CVL-I', 13: 'DAT-B', 14: 'DAT-I', 15: 'TIM-B', 16: 'TIM-I', 17: 'NUM-B', 18: 'NUM-I', 19: 'EVT-B', 20: 'EVT-I', 21: 'ANM-B', 22: 'ANM-I', 23: 'PLT-B', 24: 'PLT-I', 25: 'MAT-B', 26: 'MAT-I', 27: 'TRM-B', 28: 'TRM-I'}


In [13]:
tag_size = len(tag_to_index)
print(tag_size)

29


- 전처리 예시  

임의로 훈련 데이터 중 1번 인덱스의 문장과 레이블을 선택하고 이에 대해서 전처리를 진행해보겠습니다. 

In [14]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

sent = train_data_sentence[1]
label = train_data_label[1]

print(sent)
print(label)
print([tag_to_index[idx] for idx in label])
print(len(sent))
print(len(label))

['▶', '쿠마리', '한동수가', '말하는', "'가넷", '&', "에르덴'"]
['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
[0, 1, 2, 0, 1, 0, 1]
7
7


In [15]:
tokens = []

for one_word in sent:
    # 각 단어에 대해 subword로 분리
    # one_word = '쿠마리' ==> subword_tokens = ['쿠', '##마리']
    subword_tokens = tokenizer.tokenize(one_word)
    tokens.extend(subword_tokens)

print('BERT 토크나이저 적용 후 문장 : ', tokens)
print('레이블 : ', label)
print('레이블의 정수 인코딩 : ', [tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(tokens))
print('레이블의 길이 :', len(label))

BERT 토크나이저 적용 후 문장 :  ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블 :  ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레이블의 정수 인코딩 :  [0, 1, 2, 0, 1, 0, 1]
문장의 길이 : 16
레이블의 길이 : 7


위 결과를 보면 '쿠마리'가 '쿠'와 '##마리'로 분리되는 등 단어들이 subword로 분리되었습니다. 이제 문장의 길이가 길어지면서 레이블의 길이와 달라지게 됩니다. 따라서 레이블의 길이도 문장의 길이와 일치하도록 추가적인 처리를 진행해야 합니다. '쿠마리'의 레이블은 'PER-B'였습니다. 그렇다면 '쿠'와 '##마리'의 레이블은 어떻게 해야할까요?
<br />

이 경우 첫 번째 subword에 대해서만 기존의 레이블을 부여하고 뒤에 생겨난 subwords에 대해서는 레이블을 주지 않는 방법이 있습니다. 따라서 첫 번째 서브워드가 아닌 경우에 정수 -100을 부여하는 방식을 사용합니다.

In [16]:
tokens = []
labels_ids = []
for one_word, label_token in zip(train_data_sentence[1], train_data_label[1]):
    subword_tokens = tokenizer.tokenize(one_word)
    tokens.extend(subword_tokens)
    labels_ids.extend([tag_to_index[label_token]] + [-100] * (len(subword_tokens)-1))

print('토큰화 후 문장 : ', tokens)
print('레이블 : ', ['[PAD]' if idx==-100 else index_to_tag[idx] for idx in labels_ids])
print('레이블의 정수 인코딩 : ', labels_ids)
print('문장의 길이 :', len(tokens))
print('레이블의 길이 : ', len(labels_ids))

토큰화 후 문장 :  ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블 :  ['O', 'PER-B', '[PAD]', 'PER-I', '[PAD]', '[PAD]', 'O', '[PAD]', '[PAD]', 'PER-B', '[PAD]', '[PAD]', 'O', 'PER-B', '[PAD]', '[PAD]']
레이블의 정수 인코딩 :  [0, 1, -100, 2, -100, -100, 0, -100, -100, 1, -100, -100, 0, 1, -100, -100]
문장의 길이 : 16
레이블의 길이 :  16


위 결과에서 '레이블의 정수 인코딩'결과를 보면 '쿠'에 대해서는 PER-B에 해당하는 정수 1을 부여하였지만, '##마리'에 대해서는 -100을 부여하였습니다. 마찬가지로 기존 '한동수가'에 PER-I가 부여되어 있었으나, '한동'에는 PER-I에 해당하는 정수인 2를 부여하지만 그 뒤의 서브워드들인 '##수', '##가'에 대해서는 -100을 부여합니다.
<br />

-100을 부여하고나서는 실질적으로 학습할 때는 해당 레이블에 대해서는 학습을 무시하는 방법을 사용합니다. 정확히는 -100을 레이블에서는 패딩 토큰[PAD]로 사용합니다. 레이블에 대해서 문장의 길이를 맞추는 패딩을 진행할 때도 -100을 사용하겠습니다. 그리고 이후 손실 함수에서 -100을 무시하도록 추가 처리를 진행하겠습니다.
<br />

이제 위 과정을 한 번에 진행하고, 어텐션 마스크까지 생성하는 함수를 정의하겠습니다.

In [17]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer, pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        tokens = []
        labels_ids = []
        for one_word, label_token in zip(example, label):
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)

            labels_ids.extend([tag_to_index[label_token]] + [pad_token_id_for_label] * (len(subword_tokens) - 1))

        
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]
        
        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]

        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids

        input_id = tokenizer.convert_tokens_to_ids(tokens)

        attention_mask = [1] * len(input_id)

        padding_count = max_seq_len - len(input_id)

        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)

        token_type_id = [pad_token_id_for_segment] * max_seq_len

        label = labels_ids + ([pad_token_id_for_label] * padding_count)
        
        assert len(input_id) == max_seq_len, "ERROR with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "ERROR with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "ERROR with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "ERROR with labels length {} vs {}".format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)
    
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

        

In [18]:
x_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)
x_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 9000/9000 [00:02<00:00, 3073.20it/s]


In [20]:
print('원문 : ',train_data_sentence[0])
print('기존 레이블 : ',train_data_label[0])
print('-'*50)
print('토큰화 후 원문 : ', [tokenizer.decode([word]) for word in x_train[0][0]])
print('토큰화 후 레이블 : ', ['[PAD]' if idx==-100 else index_to_tag[idx] for idx in y_train[0]])
print('-'*50)
print('정수 인코딩 결과 : ', x_train[0][0])
print('정수 인코딩 레이블 : ', y_train[0])

원문 :  ['정은', '씨를', '힘들게', '한', '가스나그,', '가만둘', '수', '없겠죠', '.']
기존 레이블 :  ['PER-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
--------------------------------------------------
토큰화 후 원문 :  ['[CLS]', '정은', '씨', '##를', '힘들', '##게', '한', '가스', '##나', '##그', ',', '가만', '##둘', '수', '없', '##겠', '##죠', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'

In [21]:
print('Segment encoding : ', x_train[2][0])
print('Attention mask : ', x_train[1][0])

Segment encoding :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Attention mask :  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


- 모델링
<br />

이번에는 모든 입력에 대해서 출력을 수행해야 하는 Many-to-Many 문제에 해당하므로 outputs[0]을 사용하여 문제를 풀어야 합니다.
<br />

outputs[0]과 연결되는 출력층에는 레이블의 개수인 num_labels를 전달합니다. 이번에는 다중 클래스 분류 문제임에도 출력층에 소프트맥스 함수를 사용하지 않았는데, 이는 출력층에서 소프트맥스를 사용하지 않고 손실 함수에서 이를 처리하도록 하는 구현 방식을 사용하기 위함입니다.

In [22]:
class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels, kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02), name='classifier')
    
    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids = input_ids, attention_mask=attention_mask, token_type_ids = token_type_ids)

        all_output = outputs[0]
        prediction = self.classifier(all_output)

        return prediction

In [23]:
# 손실 함수
def compute_loss(labels, logits):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)

    # -100의 값을 가진 정수에 대해서는 오차를 반영하지 않도록 labels를 수정
    active_loss = tf.reshape(labels, (-1,)) != -100

    # active_loss로부터 reduced_logits와 labels를 각가 얻는다.
    reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
    labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

    return loss_fn(labels, reduced_logits)

In [24]:
model = TFBertForTokenClassification('klue/bert-base', num_labels=tag_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=compute_loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [25]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, x_test, y_test):
        self.x_test = x_test
        self.y_test = y_test
    
    def sequences_to_tags(self, label_ids, pred_ids):
        label_list = []
        pred_list = []

        for i in range(0, len(label_ids)):
            label_tag = []
            pred_tag = []

            for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
                if label_index != -100:
                    label_tag.append(index_to_tag[label_index])
                    pred_tag.append(index_to_tag[pred_index])
            
            label_list.append(label_tag)
            pred_list.append(pred_tag)
        
        return label_list, pred_list
    
    # 에포크가 끝날 때마다 실행되는 함수
    def on_epoch_end(self, epoch, logs={}):

        y_predicted = self.model.predict(self.x_test)
        y_predicted = np.argmax(y_predicted, axis=2)

        label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

        score = f1_score(label_list, pred_list, suffix=True)
        print(' - f1: {:4.2f}'.format(score * 100))
        print(classification_report(label_list, pred_list, suffix=True))

In [26]:
f1_score_report = F1score(x_test, y_test)

model.fit(x_train, y_train, epochs=3, batch_size=32, callbacks=[f1_score_report])

Epoch 1/3
282/282 [==============================] - 25s 80ms/step
 - f1: 84.65
              precision    recall  f1-score   support

         AFW       0.60      0.57      0.59       394
         ANM       0.75      0.73      0.74       701
         CVL       0.85      0.79      0.82      5758
         DAT       0.92      0.91      0.91      2521
         EVT       0.75      0.76      0.75      1094
         FLD       0.70      0.33      0.45       228
         LOC       0.84      0.85      0.84      2126
         MAT       0.50      0.08      0.14        12
         NUM       0.91      0.92      0.91      5590
         ORG       0.87      0.86      0.87      4086
         PER       0.89      0.87      0.88      4426
         PLT       0.57      0.12      0.20        34
         TIM       0.87      0.83      0.85       314
         TRM       0.76      0.64      0.70      1964

   micro avg       0.86      0.83      0.85     29248
   macro avg       0.77      0.66      0.69     29248


In [29]:
# 예측 및 평가
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer, pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks=[], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0] + [pad_token_id_for_label] * (len(subword_tokens)-1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]

        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask

        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [30]:
x_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 2506.16it/s]


In [32]:
def ner_prediction(examples, max_seq_len, tokenizer):
    examples = [sent.split() for sent in examples]
    x_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
    y_predicted = model.predict(x_pred)
    y_predicted = np.argmax(y_predicted, axis=2)

    pred_list = []
    result_list = []

    for i in range(0, len(label_masks)):
        pred_tag = []
        for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
            if label_index != -100:
                pred_tag.append(index_to_tag[pred_index])
        pred_list.append(pred_tag)

    for example, pred in zip(examples, pred_list):
        one_sample_result = []
        for one_word, label_token in zip(example, pred):
            one_sample_result.append((one_word, label_token))
        result_list.append(one_sample_result)
    
    return result_list

In [33]:
sent1 = '오리온스는 리그 최정상급 포인트가드 김동훈을 앞세우는 빠른 공수전환이 돋보이는 팀이다'
sent2 = '하이신사에 속한 섬들도 위로 솟아 있는데 타인은 살고 있어요'

In [34]:
test_sample = [sent1, sent2]

result_list = ner_prediction(test_sample, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 2/2 [00:00<00:00, 3126.58it/s]


1/1 [==============================] - 0s 185ms/step


In [35]:
result_list

[[('오리온스는', 'ORG-B'),
  ('리그', 'O'),
  ('최정상급', 'O'),
  ('포인트가드', 'CVL-B'),
  ('김동훈을', 'PER-B'),
  ('앞세우는', 'O'),
  ('빠른', 'O'),
  ('공수전환이', 'O'),
  ('돋보이는', 'O'),
  ('팀이다', 'O')],
 [('하이신사에', 'LOC-B'),
  ('속한', 'O'),
  ('섬들도', 'O'),
  ('위로', 'O'),
  ('솟아', 'O'),
  ('있는데', 'O'),
  ('타인은', 'O'),
  ('살고', 'O'),
  ('있어요', 'O')]]